In [15]:
import numpy as np
import torch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#from models.BivariateHMM import BivariateHMM
from models.CopulaHMM import CopulaHMM
from utils.Plots import plotEPS_with_states,plotEPS_distribution

In [3]:
DATA_DIR="data/"
MAX_HIDDEN_STATES=5
MIN_HIDDEN_STATES=2

In [4]:
data = pd.read_csv(f"{DATA_DIR}hulls_df_matchday2_reduced.csv")
data = data.dropna()

events=pd.read_csv(f"{DATA_DIR}matchday2_events.csv")
goals_info=events[events["Subtype"].isin(["ON TARGET-GOAL","HEAD-ON TARGET-GOAL","WOODWORK-GOAL"])]
home_goals=goals_info[goals_info["Team"]=="Home"]
away_goals=goals_info[goals_info["Team"]=="Away"]
shots_info=events[events["Type"]=="SHOT"]
home_shot=shots_info[shots_info["Team"]=="Home"]
away_shot=shots_info[shots_info["Team"]=="Away"]


sequence_XY = torch.tensor(data[["HomeHull","AwayHull"]].values/100)

## Quantitative comparison

In [125]:
AIC_list=[]
for state in range(2,MAX_HIDDEN_STATES+1):
    posterior = torch.load(f"parameters/CopulaHMM_matchday2_{state}states.pt")
    model=CopulaHMM.from_posterior(posterior)
    AIC_list.append(model.AIC(sequence_XY).item())

In [126]:
for state,aic in enumerate(AIC_list):
    print(f"AIC of the model with {state+MIN_HIDDEN_STATES} hidden states-> {aic}")

AIC of the model with 2 hidden states-> 18766.37109375
AIC of the model with 3 hidden states-> 17957.654296875
AIC of the model with 4 hidden states-> 17971.048828125
AIC of the model with 5 hidden states-> 17797.033203125


## Qualitative comparison

In [17]:
colors_list=["#FD8033","#0DC2B7","#DAC11E","#D964DC","#37A010"]

In [19]:
def plotEPS_hist(data,class_colors):
    fig, ax = plt.subplots(1, 2, figsize=(14, 5))  # Adjusted figsize for better visualization

    sns.histplot(x=data["HomeHull"]*100, hue=data["HomeMLS"], kde=True, palette=class_colors, alpha=0.2, ax=ax[0])
    ax[0].set_xlabel("Convex Hull Area", fontsize=12, fontweight='normal')
    ax[0].set_ylabel("Count", fontsize=12, fontweight='normal')
    ax[0].set_title("Home Team Convex Hull Area by State", fontsize=15, fontweight='bold')

    sns.histplot(x=data["AwayHull"], hue=data["AwayMLS"], kde=True, palette=class_colors, alpha=0.2, ax=ax[1])
    ax[1].set_xlabel("Convex Hull Area", fontsize=12, fontweight='normal')
    ax[1].set_ylabel("Count", fontsize=12, fontweight='normal')
    ax[1].set_title("Away Team Convex Hull Area by State", fontsize=15, fontweight='bold')

    plt.subplots_adjust(hspace=0.45)
    plt.close(fig)
    return fig

In [22]:
for state in range(2,MAX_HIDDEN_STATES+1):
    posterior = torch.load(f"parameters/CopulaHMM_matchday2_{state}states.pt")
    #print(posterior)
    model=CopulaHMM.from_posterior(posterior)
    MLS=model.viterbi(sequence_XY)
    # data[f"State_{state}"]=MLS.numpy()
    class_colors = {k: colors_list[k] for k in range(state)}
    print(class_colors)
    # p1=plotEPS_distribution(data,class_colors)
    # p2=plotEPS_with_states(data,home_goals,away_goals,home_shot,away_shot,class_colors)
    # p3 = plotEPS_hist(data,class_colors)
    # # -----Save-----
    # #p1.savefig(f"plots/EPS_distribution_matchday2_{HIDDEN_STATES}states.png",dpi=350)
    # #p2.savefig(f"plots/EPS_with_states_matchday2_{HIDDEN_STATES}states.png",dpi=350)
    # p3.savefig(f"plots/EPS_hist_matchday2_{state}states.png",dpi=350)

{'probs_initial': tensor([0.9140, 0.0860], requires_grad=True), 'probs_x': tensor([[0.9536, 0.0464],
        [0.0502, 0.9498]], requires_grad=True), 'probs_alpha1': tensor([ 6.3268, 13.0156], requires_grad=True), 'probs_beta1': tensor([0.8216, 1.3320], requires_grad=True), 'probs_alpha2': tensor([22.3060, 10.9269], requires_grad=True), 'probs_beta2': tensor([1.8945, 1.6462], requires_grad=True), 'theta': tensor([0.4305, 0.5557], requires_grad=True)}
{0: '#FD8033', 1: '#0DC2B7'}
{'probs_initial': tensor([0.9138, 0.0224, 0.0638], requires_grad=True), 'probs_x': tensor([[0.9138, 0.0471, 0.0391],
        [0.0171, 0.9499, 0.0330],
        [0.0355, 0.0348, 0.9297]], requires_grad=True), 'probs_alpha1': tensor([ 7.9685, 23.8327, 13.2033], requires_grad=True), 'probs_beta1': tensor([1.5057, 2.2917, 1.5057], requires_grad=True), 'probs_alpha2': tensor([12.1551, 10.7805, 35.6327], requires_grad=True), 'probs_beta2': tensor([1.3643, 1.5773, 2.7527], requires_grad=True), 'theta': tensor([1.1439, 0